In [35]:
import numpy as np
from numpy import pi
# importing Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_ibm_provider import IBMProvider

In [36]:
api_token='IBM TOKEN'

IBMProvider.save_account(api_token, overwrite=True)
provider = IBMProvider()
print("Available backends:", provider.backends())

Available backends: [<IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_sherbrooke')>]


In [37]:
from numpy.random import randint

In [45]:
num_qubits=4
alice_basis = np.random.randint(2,size=num_qubits)
alice_state = np.random.randint(2,size=num_qubits)

bobs_basis = np.random.randint(2,size=num_qubits)

print(f"alice_state {alice_state}")
print(f"alice_basis {alice_basis}")
print(f"alice_basis {bobs_basis}")


alice_state [1 1 0 1]
alice_basis [0 1 0 1]
alice_basis [0 1 1 0]


In [46]:
def apply_protocol(state,basis,measurement_basis):
    nc = len(state)
    qc = QuantumCircuit(nc)
    for idx in range(nc):
        if(state[idx]==1):
            qc.x(idx)
    for idx in range(nc):
        if(basis[idx]==1):
            qc.h(idx)
    for idx in range(nc):
        if(measurement_basis[idx]==1):
            qc.h(idx)
    qc.measure_all()
    return qc

In [47]:
qc = apply_protocol(alice_state,alice_basis,bobs_basis)



In [43]:
aer_sim = Aer.get_backend('aer_simulator')
shots = 2048
t_qpe = transpile(qc.reverse_bits(), aer_sim)
results = aer_sim.run(t_qpe, shots=shots).result()
bob_output = results.get_counts().most_frequent()


In [48]:
backends = provider.backends(filters=lambda b: b.status().operational and not b.configuration().simulator)
least_busy_backend = min(backends, key=lambda b: b.status().pending_jobs)
shots = 2048
transpiled_qc = transpile(qc.reverse_bits(), least_busy_backend, optimization_level=3)
job = least_busy_backend.run(transpiled_qc, shots=shots)
bob_output = job.result().get_counts().most_frequent()


IBMJobInvalidStateError: 'Unable to retrieve result for job cvmf668qwj7g008y3zk0. Job was cancelled.'

In [31]:
enc_key_alice=""
enc_key_bob=""
for i in range(num_qubits):
    if(alice_basis[i]==bobs_basis[i]):
        enc_key_bob += str(bob_output[i])
        enc_key_alice += str(alice_state[i])

print(f" alice key: {enc_key_alice}")
print(f" bob   key: {enc_key_bob}")

 alice key: 00011
 bob   key: 00011
